In [1]:
# Colab: Previsão simples de vendas + relatório Excel

# Importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from datetime import timedelta
import matplotlib.pyplot as plt
from io import BytesIO
from google.colab import files
import openpyxl
from openpyxl.chart import LineChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows

In [4]:
# === 1. Upload dos arquivos ===
print("Faça upload do arquivo com dados passados (csv/xlsx):")
uploaded_passado = files.upload()
print("\nFaça upload do arquivo com dados atuais para comparação (csv/xlsx):")
uploaded_atual = files.upload()

# Pega nome dos arquivos
file_passado = list(uploaded_passado.keys())[0]
file_atual = list(uploaded_atual.keys())[0]

Faça upload do arquivo com dados passados (csv/xlsx):


Saving DADOS_2023_2024.csv to DADOS_2023_2024 (1).csv

Faça upload do arquivo com dados atuais para comparação (csv/xlsx):


Saving DEMANDAS_REAIS.csv to DEMANDAS_REAIS (1).csv


In [9]:
# === 2. Leitura dos arquivos ===
df_passado = pd.read_csv(file_passado)
df_atual = pd.read_csv(file_atual)

print("📋 Visualização inicial dos dados:-df_passado")
df_passado.head(3)


📋 Visualização inicial dos dados:-df_passado


,DT_VENDA,CD_PRODUTO,DS_PRODUTO,UND_MED,QTD_SAIDA
0,22/02/23 00:00:00,27090,REFRIG COCA COLA PET 2L,UN,135
1,26/02/23 00:00:00,225,COXA/SOBREC C/DORSO PCTAO KG,KG,99
2,10/05/23 00:00:00,225,COXA/SOBREC C/DORSO PCTAO KG,KG,388


In [10]:
print(f"Passado: {df_passado.shape[0]} registros")
print(f"Atual: {df_atual.shape[0]} registros")


print("📋 Visualização inicial dos dados:-df_atual")
df_atual.head(3)

Passado: 7265 registros
Atual: 50 registros
📋 Visualização inicial dos dados:-df_atual


,ANOMES_VENDA,CD_PRODUTO,DS_PRODUTO,QTD_SAIDA
0,202503,53056,SABON PALMOLIVE 85G HIDR INTENSIVA,877
1,202504,30812,OVO GAL LOCKS BJ C/30 PLAST,5403
2,202505,16871,DETERG LIMPOL NEUTRO FR 500ML,2751


In [11]:
# === 3. Função para agregação por período ===
def agrega_periodo(df, freq):
    # freq: 'D' = diário, 'W' = semanal, 'M' = mensal
    df_agr = df.copy()
    df_agr['DT_VENDA'] = pd.to_datetime(df_agr['DT_VENDA'])
    df_agr['periodo'] = df_agr['DT_VENDA'].dt.to_period(freq).dt.start_time
    df_agr = df_agr.groupby(['CD_PRODUTO', 'periodo'])['QTD_SAIDA'].sum().reset_index()
    return df_agr.rename(columns={'periodo': 'DT_VENDA'})

# === 4. Treina e prevê para um produto ===
def treina_preve(df_train, df_test):
    # Converte data para dias desde o mínimo
    df_train = df_train.copy()
    df_test = df_test.copy()
    df_train['dias'] = (df_train['DT_VENDA'] - df_train['DT_VENDA'].min()).dt.days
    df_test['dias'] = (df_test['DT_VENDA'] - df_train['DT_VENDA'].min()).dt.days

    X_train = df_train[['dias']]
    y_train = df_train['QTD_SAIDA']
    X_test = df_test[['dias']]

    # Modelos
    lr = LinearRegression()
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    lr.fit(X_train, y_train)
    rf.fit(X_train, y_train)

    # Previsões
    df_test['pred_lr'] = lr.predict(X_test)
    df_test['pred_rf'] = rf.predict(X_test)

    return df_test, lr, rf

# === 5. Calcula estoques mínimos, ideais e máximos ===
def calcula_estoque(previsao):
    # Estoque mínimo = 70% da média da previsão
    # Estoque ideal = média da previsão
    # Estoque máximo = 130% da média da previsão
    media = previsao['QTD_SAIDA'].mean()
    return {
        'estoque_min': media * 0.7,
        'estoque_ideal': media,
        'estoque_max': media * 1.3
    }

# === 6. Processo completo e relatório ===
def processo_geral(df_passado, df_atual, freq):
    print(f"\n--- Processando granularidade: {freq} ---")
    df_train = agrega_periodo(df_passado, freq)
    df_test = agrega_periodo(df_atual, freq)

    produtos = df_train['CD_PRODUTO'].unique()
    resultado = []

    for p in produtos:
        treino_p = df_train[df_train['CD_PRODUTO'] == p]
        teste_p = df_test[df_test['CD_PRODUTO'] == p]

        if len(treino_p) < 3 or len(teste_p) == 0:
            print(f"Produto {p}: dados insuficientes para treino ou teste.")
            continue

        previsao, lr, rf = treina_preve(treino_p, teste_p)

        estoque = calcula_estoque(treino_p)
        previsao['estoque_min'] = estoque['estoque_min']
        previsao['estoque_ideal'] = estoque['estoque_ideal']
        previsao['estoque_max'] = estoque['estoque_max']

        # Erros
        mae_lr = mean_absolute_error(previsao['QTD_SAIDA'], previsao['pred_lr'])
        mae_rf = mean_absolute_error(previsao['QTD_SAIDA'], previsao['pred_rf'])

        print(f"Produto {p} - MAE LR: {mae_lr:.2f} / MAE RF: {mae_rf:.2f}")

        resultado.append(previsao.assign(CD_PRODUTO=p))

    if len(resultado) == 0:
        print("Nenhum produto processado.")
        return None

    df_res = pd.concat(resultado)

    return df_res

# === 7. Rodar para todas granularidades e salvar em Excel ===

writer = pd.ExcelWriter('relatorio_vendas.xlsx', engine='openpyxl')

for freq, nome in [('D', 'Diaria'), ('W', 'Semanal'), ('M', 'Mensal')]:
    res = processo_geral(df_passado, df_atual, freq)
    if res is not None:
        res.to_excel(writer, sheet_name=nome, index=False)

writer.save()
print("\nRelatório Excel criado: relatorio_vendas.xlsx")

# === 8. Download do arquivo ===
files.download('relatorio_vendas.xlsx')


--- Processando granularidade: D ---


/tmp/ipython-input-11-989902434.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_agr['DT_VENDA'] = pd.to_datetime(df_agr['DT_VENDA'])


KeyError: 'DT_VENDA'